## Ingredient Phrase Model

This program will create a model that is designed to separately identify food name, quantity and other information as Named Entity Recognition tags from a word ingredient list.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import re

# Generate training data using NY Times ingredient phrase tagger
from ingredient_phrase_tagger.training.cli import Cli
from ingredient_phrase_tagger.training import utils, reshape

from sklearn.model_selection import train_test_split

# Model libraries
from tagger_model import *

from IPython.core.debugger import set_trace

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Some default parameters
n_word_embedding_nodes=300
n_tag_embedding_nodes=150
n_RNN_nodes=400
n_dense_nodes=200

dataPath = '../data'

In [3]:
filename = 'cleaned_nyt_ingred_data.pkl'

# reshape.read_and_save_raw_data(dataPath, filename)
cleaned_dat = pd.read_pickle(os.path.join(dataPath, filename))

# clean up tags to remove prefixes so that there are less tags to predict
cleaned_dat['tags'] = [[re.sub(r'[B|I]-', '', tag) for tag in tags] for tags in cleaned_dat.tags]

In [4]:
train, test = train_test_split(cleaned_dat, test_size = .2, random_state=10)

In [5]:
# Create lexicon
lexicon = lexiconTransformer(words_min_freq=2, unknown_tag_token='OTHER', saveNamePrefix='Ingred_mod')

lexicon.fit(train.sents, train.tags)

train['sent_indx'], train['tag_indx'] = lexicon.transform(train.sents, train.tags)

# Get length of longest sequence
max_seq_len = get_max_seq_len(train['sent_indx'])

#Add one to max length for offsetting sequence by 1
train_padded_words = pad_idx_seqs(train['sent_indx'], 
                                  max_seq_len + 1) 

train_padded_tags = pad_idx_seqs(train['tag_indx'],
                                 max_seq_len + 1)

# Shift tags by 1 for training since we should use tag of previous 
# iteration in next iteration
shifted_train_padded_tags = np.insert(train_padded_tags, 0, 1, axis=1)[:, :-1]

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
# Convert to one-hot vector encoding for y
# train_y = [to_categorical(i, num_classes=len(lexicon.tags_lexicon) + 1) for i in train_padded_tags]

In [7]:
mod_save_name = 'ingredient_model_clean_tags_crf_wordOnly'
crf_mod = True

In [ ]:
ingredient_model = run_training_model(train_padded_words, train_padded_tags, 
                                      train_padded_tags, mod_save_name, lexicon, crf=crf_mod,
                                      print_summary=True, batch_size=256, epochs=200,
                                      n_word_embedding_nodes=n_word_embedding_nodes,
                                      n_tag_embedding_nodes=n_tag_embedding_nodes,
                                      n_RNN_nodes=n_RNN_nodes, 
                                      n_dense_nodes=n_dense_nodes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input_layer (InputLayer (None, 56)                0         
_________________________________________________________________
word_embedding_layer (Embedd (None, 56, 300)           1699800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 56, 800)           1682400   
_________________________________________________________________
dense_layer (TimeDistributed (None, 56, 200)           160200    
_________________________________________________________________
output_layer (CRF)           (None, 56, 7)             1470      
Total params: 3,543,870
Trainable params: 3,543,870
Non-trainable params: 0
_________________________________________________________________
Train on 114600 samples, validate on 28650 samples
Epoch 1/200
114600/114600 [==============================] - 131s 1ms/step - loss


Epoch 00032: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 33/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3872 - acc: 0.9417 - val_loss: 21.6948 - val_acc: 0.8393

Epoch 00033: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 34/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3860 - acc: 0.9421 - val_loss: 21.7012 - val_acc: 0.8418

Epoch 00034: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 35/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3844 - acc: 0.9438 - val_loss: 21.7105 - val_acc: 0.8417

Epoch 00035: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 36/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3835 - acc: 0.9444 - val_loss: 21.7122 - val_acc: 0.8376

Epoch 00036: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 37/

114600/114600 [==============================] - 129s 1ms/step - loss: 21.3660 - acc: 0.9539 - val_loss: 21.8017 - val_acc: 0.8407

Epoch 00069: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 70/200
114600/114600 [==============================] - 128s 1ms/step - loss: 21.3649 - acc: 0.9548 - val_loss: 21.7998 - val_acc: 0.8381

Epoch 00070: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 71/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3654 - acc: 0.9545 - val_loss: 21.8039 - val_acc: 0.8379

Epoch 00071: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 72/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3648 - acc: 0.9544 - val_loss: 21.8037 - val_acc: 0.8361

Epoch 00072: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 73/200
114600/114600 [==============================] - 128s 1ms/step - loss: 21.3648 - acc: 0.9543

114600/114600 [==============================] - 129s 1ms/step - loss: 21.3584 - acc: 0.9571 - val_loss: 21.8513 - val_acc: 0.8356

Epoch 00141: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 142/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3583 - acc: 0.9570 - val_loss: 21.8459 - val_acc: 0.8392

Epoch 00142: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 143/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3581 - acc: 0.9575 - val_loss: 21.8528 - val_acc: 0.8402

Epoch 00143: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 144/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3582 - acc: 0.9571 - val_loss: 21.8500 - val_acc: 0.8373

Epoch 00144: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 145/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3585 - acc: 0.

114600/114600 [==============================] - 128s 1ms/step - loss: 21.3574 - acc: 0.9569 - val_loss: 21.8502 - val_acc: 0.8363

Epoch 00177: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 178/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3573 - acc: 0.9576 - val_loss: 21.8555 - val_acc: 0.8373

Epoch 00178: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 179/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3575 - acc: 0.9575 - val_loss: 21.8526 - val_acc: 0.8387

Epoch 00179: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 180/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3573 - acc: 0.9575 - val_loss: 21.8535 - val_acc: 0.8387

Epoch 00180: saving model to models/ingredient_model_clean_tags_crf_wordOnly.hdf5
Epoch 181/200
114600/114600 [==============================] - 129s 1ms/step - loss: 21.3571 - acc: 0.

In [8]:
test['sent_indx'], test['tag_indx'] = lexicon.transform(test.sents, test.tags)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
test_mod = create_test_model(mod_save_name, lexicon, crf=crf_mod, 
                             n_word_embedding_nodes=n_word_embedding_nodes,
                             n_tag_embedding_nodes=n_tag_embedding_nodes,
                             n_RNN_nodes=n_RNN_nodes, 
                             n_dense_nodes=n_dense_nodes)

In [10]:
preds = predict_new_tag(test_mod, test, lexicon)

In [11]:
evaluate_model(preds, test, print_sample=True)

SENTENCE:	1/2	cup	heavy	cream
PREDICTED:	QTY	UNIT	NAME	NAME
GOLD:		QTY	UNIT	NAME	NAME
CORRECT:	True	True	True	True 


SENTENCE:	1/4	cup	low-fat	vanilla	yogurt
PREDICTED:	QTY	UNIT	NAME	NAME	NAME
GOLD:		QTY	UNIT	COMMENT	COMMENT	NAME
CORRECT:	True	True	False	False	True 


SENTENCE:	1/3	cup	extra-virgin	olive	oil
PREDICTED:	OTHER	UNIT	COMMENT	NAME	NAME
GOLD:		OTHER	UNIT	COMMENT	NAME	NAME
CORRECT:	True	True	True	True	True 


SENTENCE:	Coarse	salt	or	kosher	salt
PREDICTED:	COMMENT	NAME	OTHER	NAME	NAME
GOLD:		COMMENT	NAME	COMMENT	COMMENT	NAME
CORRECT:	True	True	False	False	True 


SENTENCE:	2	teaspoons	minced	ginger	root
PREDICTED:	QTY	UNIT	COMMENT	NAME	NAME
GOLD:		QTY	UNIT	COMMENT	NAME	NAME
CORRECT:	True	True	True	True	True 


SENTENCE:	1	ounce	Carpano	Antica	Formula	vermouth
PREDICTED:	QTY	UNIT	NAME	COMMENT	NAME	NAME
GOLD:		QTY	UNIT	NAME	NAME	NAME	NAME
CORRECT:	True	True	True	False	True	True 


SENTENCE:	2\ntablespoons	olive	oil
PREDICTED:	OTHER	NAME	NAME
GOLD:		OTHER	NAME	NAME
CORRECT:	Tru

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ACCURACY: 0.689
PRECISION: 0.721
RECALL: 0.689
F1: 0.688


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
